In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *

import pickle
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

import warnings
warnings.filterwarnings("ignore")

<h1>Configuration Stuff Here</h1>

In [ ]:
m = 200

ifSave = False
savePath = "../plot_files/05Mar2024_m"+str(m)+"_systematics/"

In [ ]:
unisim_list = [ 'slc_xsec_unisim_DecayAngMEC', 
               'slc_xsec_unisim_ThetaDelta2NRad',
               'slc_xsec_unisim_Theta_Delta2Npi', 
               'slc_xsec_unisim_VecFFCCQEshape',]

multisigma_list = [
        'slc_xsec_multisigma_CoulombCCQE',
       'slc_xsec_multisigma_NonRESBGvbarnCC1pi',
       'slc_xsec_multisigma_NonRESBGvbarnCC2pi',
       'slc_xsec_multisigma_NonRESBGvbarnNC1pi',
       'slc_xsec_multisigma_NonRESBGvbarnNC2pi',
       'slc_xsec_multisigma_NonRESBGvbarpCC1pi',
       'slc_xsec_multisigma_NonRESBGvbarpCC2pi',
       'slc_xsec_multisigma_NonRESBGvbarpNC1pi',
       'slc_xsec_multisigma_NonRESBGvbarpNC2pi',
       'slc_xsec_multisigma_NonRESBGvnCC1pi',
       'slc_xsec_multisigma_NonRESBGvnCC2pi',
       'slc_xsec_multisigma_NonRESBGvnNC1pi',
       'slc_xsec_multisigma_NonRESBGvnNC2pi',
       'slc_xsec_multisigma_NonRESBGvpCC1pi',
       'slc_xsec_multisigma_NonRESBGvpCC2pi',
       'slc_xsec_multisigma_NonRESBGvpNC1pi',
       'slc_xsec_multisigma_NonRESBGvpNC2pi', 
       'slc_xsec_multisigma_NormCCMEC',
       'slc_xsec_multisigma_NormNCMEC', 
       'slc_xsec_multisigma_RDecBR1eta',
       'slc_xsec_multisigma_RDecBR1gamma', 
       'slc_xsec_multisigma_RPA_CCQE',
       'slc_xsec_multisigma_NormNCCOH', 
       'slc_xsec_multisigma_NormCCCOH',
]

<h1> Read in PKL Dataframe </h1>

<h3>HNL</h3>

In [ ]:
hnl_path = "../pkl_files/v3/df_m"+str(m)+"_v3_hnl.pkl"

file = open(hnl_path, 'rb')
df_hnl = pickle.load(file)
file.close
del hnl_path

In [ ]:
df_hnl = df_hnl[["run", "subrun", "event", "slc_idx", "scale_pot", "simU", "scaledU", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h3>TPC Neutrino</h3>

In [ ]:
nu_path = "../pkl_files/v3/df_m"+str(m)+"_v3_nu.pkl"

file = open(nu_path, 'rb')
df_nu = pickle.load(file)
file.close
del nu_path

In [ ]:
df_nu = df_nu[["run", "subrun", "event", "slc_idx", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h3>Intime Cosmics</h3>

In [ ]:
cos_path = "../pkl_files/v3/df_m"+str(m)+"_v3_cos.pkl"

file = open(cos_path, 'rb')
df_cos = pickle.load(file)
file.close
del cos_path

In [ ]:
df_cos = df_cos[["run", "subrun", "event", "slc_idx", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h1> Load Flux Systematics</h1>

<h3> HNL </h3>

In [ ]:
dfw_hnl = pd.DataFrame()

print(len(df_hnl))

for i in range(0, 3):  
    hnl_path = "../pkl_files/v3/hnl_m"+str(m)+"_50k_v3_flxw_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')   
    df = pickle.load(file)
    df = df[['run','subrun','event','slc_idx','slc_flux_weight_total']]

    print(df[(df['run'] == 1) & (df['subrun']==790) & (df['event']==39)])
    
    df = df.merge(df_hnl, how='inner', on=['run','subrun','event','slc_idx'])
    
    print(len(df))
    print(len(df_hnl))
    dfw_hnl = pd.concat((dfw_hnl, df), ignore_index=True)
    file.close()
    
del df_hnl

print(len(dfw_hnl))

<h3> TPC Neutrino </h3>

In [ ]:
dfw_nu = pd.DataFrame()

print(len(df_nu))

for i in range(0, 13):
    nu_path = "../pkl_files/v3/rockbox_100k_v3_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_idx'])
    
    dfw_nu = pd.concat((dfw_nu, df), ignore_index=True)
    file.close()
    
del df_nu

print(len(dfw_nu))

<h1>Manipulate Dataframe</h1>

<h3>Flux Weights </h3>

In [ ]:
def make_df_flxw(df):
    
    #explode array into columns
    df_flxw = pd.DataFrame(df['slc_flux_weight_total'].tolist(),index=df.index).add_prefix('flxw_')
    
    #drop column
    df = df.drop(columns='slc_flux_weight_total')

    #merge 2 arrays together
    df = pd.concat([df, df_flxw], axis = 1)
    
    #add nominal flux weight = 1
    df['flxw_nom'] = 1
    
    del df_flxw
    return df

In [ ]:
dfw_hnl = make_df_flxw(dfw_hnl)
dfw_nu = make_df_flxw(dfw_nu)

In [ ]:
#Check for NAN for cosmic slice
nan_df = dfw_hnl[dfw_hnl.flxw_0 != dfw_hnl.flxw_0]
nan_df

In [ ]:
#fill NAN with 1
dfw_hnl = dfw_hnl.fillna(1)
dfw_hnl

<h3>Neutrino Cross Section Systematics</h3>

In [ ]:
def check_unisim(row):
    val = 1
    if len(row) > 0:
        val = row[0]
        if val == -9999:
            val = 1
    return val

In [ ]:
def check_multisigma_neg1(row):
    val = 1
    if len(row) > 0:
        val = row[0]
        if val == -9999:
            val = 1
    return val

In [ ]:
def check_multisigma_pos1(row):
    val = 1
    if len(row) > 0:
        val = row[1]
        if val == -9999:
            val = 1
    return val

In [ ]:
def make_df_xsecw(df):
    
    for name in unisim_list:
        df[name] = df[name].apply(lambda row: check_unisim(row))
    
    for name in multisigma_list:
        df[name+'_neg1'] = df[name].apply(lambda row: check_multisigma_neg1(row))
        df[name+'_pos1'] = df[name].apply(lambda row: check_multisigma_pos1(row))
        df = df.drop([name], axis = 1)
    
    #explode array into columns
    df_xsecw = pd.DataFrame(df['slc_xsec_multisim_total'].tolist(),index=df.index).add_prefix('xsecw_')
    
    #drop column
    df = df.drop(columns='slc_xsec_multisim_total')

    #merge 2 arrays together
    df = pd.concat([df, df_xsecw], axis = 1)
    
    #add nominal flux weight = 1
    df['xsec_nom'] = 1
    
    #explode array into columns
    df_g4w = pd.DataFrame(df['slc_geant4_multisim_reinteractions'].tolist(),index=df.index).add_prefix('g4w_')
    
    #drop column
    df = df.drop(columns='slc_geant4_multisim_reinteractions')

    #merge 2 arrays together
    df = pd.concat([df, df_g4w], axis = 1)
    
    #add nominal flux weight = 1
    df['g4_nom'] = 1
    
    return df


In [ ]:
dfw_nu = make_df_xsecw(dfw_nu)

In [ ]:
#fill cosmic weight with 1
dfw_nu = dfw_nu.fillna(1)
dfw_nu

In [ ]:
for name in multisigma_list:
    print()
    print(dfw_nu[name + "_pos1"])
    print(dfw_nu[name + "_neg1"])

<h1>SAVE THIS DATAFRAME</h1>

In [ ]:
dfw_hnl.to_pickle("../pkl_files/v3/df_hnl_m"+str(m)+"_v3_weight.pkl", protocol = 5)
dfw_nu.to_pickle("../pkl_files/v3/df_nu_m"+str(m)+"_v3_weight.pkl", protocol = 5)
df_cos.to_pickle("../pkl_files/v3/df_cos_m"+str(m)+"_v3_weight.pkl", protocol = 5)

<h1> START FROM HERE!!! LOAD PKL FILE WITH FLX WEIGHT ADDED </h1>

<h3>HNL</h3>

In [ ]:
file = open("../pkl_files/v3/df_hnl_m"+str(m)+"_v3_weight.pkl", 'rb')
df_hnl = pickle.load(file)
file.close()

<h3>TPC Neutrino</h3>

In [ ]:
file = open("../pkl_files/v3/df_nu_m"+str(m)+"_v3_weight.pkl", 'rb')
df_nu = pickle.load(file)
file.close()

<h3>Intime Cosmics</h3>

In [ ]:
file = open("../pkl_files/v3/df_cos_m"+str(m)+"_v3_weight.pkl", 'rb')
df_cos = pickle.load(file)
file.close()

<h3> Configure some plots stuff </h3> 

In [ ]:
xmin = 0
xmax = 19
xnbin = 19

hnl_ymin = 0
hnl_ymax = 1200

nu_ymin = 0
nu_ymax = 60

<h3> Organise Some Stuff</h3>

In [ ]:
hnl_dict = {}
nu_dict = {}

<h1> Make Beam Bucket - Post PID</h1>

In [ ]:
true_counts = 1
start_counts = 1
simU = 1

if m == 140:
    simU = 1e-5
    scaleHNLPlot = 3.7
    true_counts = 32240.456491241213
    start_counts = 30011.11138541757
    
if m == 200:
    simU=1e-7
    scaleHNLPlot = 80
    true_counts = 1491.8413161003443
    start_counts = 1372.3745440402568
    
if m == 240:
    simU = 1e-7
    scaleHNLPlot = 17.2
    true_counts = 6935.57810442791
    start_counts = 6380.176454423673

In [ ]:
plotU = df_hnl['scaledU'].unique()[0]

true_counts = true_counts * scaleHNLPlot
start_counts =  start_counts * scaleHNLPlot

print("Scale Umu by " + str(scaleHNLPlot))
print("Scaled Umu = " + str(plotU))
print("Scaled true counts = " + str(true_counts))
print("Scaled start counts = " + str(start_counts))

In [ ]:
width = 18.936

hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    xmin = xmin, xmax = xmax, xnbin = xnbin,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )
plt.show()

<h1>Statistics Uncertainty</h1>

<h3>HNL</h3>

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

pltdf = df_hnl['mod_t']
weights = None

#-----------------------------------------------------------------#
hnl_stat_noScale, bins, _ = ax1.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["Flamingo"],
                            linestyle = "-",
                            linewidth = 2,
                            label = "M = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU))
                        )

ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax)

#-----------------------------------------------------------------#
fig.tight_layout()

plt.show()

In [ ]:
#No Scale
print("\nprescale: entries per bin")
print(hnl_stat_noScale)

print("\nprescale: sqrt(entries) per bin")
hnl_stat_noScale_err = np.sqrt(hnl_stat_noScale)
print(hnl_stat_noScale_err)

print("\nprescale: fractional error = sqrt(entries) per bin/ entries per bin")
hnl_stat_rel_err = hnl_stat_noScale_err / hnl_stat_noScale
print(hnl_stat_rel_err)

In [ ]:
#This is the smart way
cov = np.diag(hnl_stat_noScale) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
fractional = cov / np.outer(hnl_stat_noScale, hnl_stat_noScale) # N / (N * N)
fractional_err = np.sqrt(np.diag(fractional)) # sqrt(N)
fractional_err

In [ ]:
hnl_stat_noScale_plot = np.insert(hnl_stat_noScale, 0, 0)

bins_mid = np.convolve(bins, [0.5, 0.5], "valid")

In [ ]:
hnl_dict['hnl_cv'] = hnl_stat_noScale
hnl_dict['hnl_cv_plot'] = hnl_stat_noScale_plot

hnl_dict['bins'] = bins
hnl_dict['bins_mid'] = bins_mid

hnl_dict['neg_stat_err']= hnl_stat_noScale_err
hnl_dict['pos_stat_err']= hnl_stat_noScale_err

hnl_dict

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

#-----------------------------------------------------------------#
#nStat NoScale
ax1.step(bins, hnl_stat_noScale_plot
         , color = col_dict['Flamingo']
         , label =  "M = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU))
        )

ax1.errorbar(bins_mid, hnl_stat_noScale, hnl_stat_noScale_err,
            ls='none',
            color = col_dict['Mauve'],
            capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#

ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax)

#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_statistics_error.png"), dpi=200)
    
plt.show()

<h3>Neutrino</h3>

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

pltdf = df_nu['mod_t']
weights = None

#-----------------------------------------------------------------#
nu_stat_noScale, bins, _ = ax1.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["Flamingo"],
                            linestyle = "-",
                            linewidth = 2,
                            label = "Neutrinos"
                        )

ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(nu_ymin, nu_ymax)

#-----------------------------------------------------------------#
fig.tight_layout()

plt.show()

In [ ]:
#No Scale
print("\nprescale: entries per bin")
print(nu_stat_noScale)

print("\nprescale: sqrt(entries) per bin")
nu_stat_noScale_err = np.sqrt(nu_stat_noScale)
print(nu_stat_noScale_err)

print("\nprescale: fractional error = sqrt(entries) per bin/ entries per bin")
nu_stat_rel_err = nu_stat_noScale_err / nu_stat_noScale
#fill nan with 0
nu_stat_rel_err = np.nan_to_num(nu_stat_rel_err, nan=0)
print(nu_stat_rel_err)

In [ ]:
nu_stat_noScale_plot = np.insert(nu_stat_noScale, 0, 0)


In [ ]:
nu_dict['nu_cv'] = nu_stat_noScale
nu_dict['nu_cv_plot'] = nu_stat_noScale_plot

nu_dict['bins'] = bins
nu_dict['bins_mid'] = bins_mid

nu_dict['neg_stat_err']= nu_stat_noScale_err
nu_dict['pos_stat_err']= nu_stat_noScale_err

nu_dict

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

#-----------------------------------------------------------------#
#nStat NoScale
ax1.step(bins, nu_stat_noScale_plot
         , color = col_dict['Flamingo']
         , label =  "Neutrinos"
        )

ax1.errorbar(bins_mid, nu_stat_noScale, nu_stat_noScale_err,
            ls='none',
            color = col_dict['Mauve'],
            capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#

ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, 60)

#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_statistics_error.png"), dpi=200)
    
plt.show()

<h1>Flux Weights</h1>

<h3>Calculate Median and Value at 68 percentile</h3>

In [ ]:
def get_median_68_quantile(n_cv, n_arr):
    print("CV array")
    n_cv = np.array(n_cv)
    print(n_cv)

    #check shape
    print("Weight array shape")
    n_arr = np.array(n_arr)
    print(n_arr.shape)

    print('\n Median')
    print(np.median(n_arr, axis=0))
    median = np.median(n_arr, axis=0)
    
    print('\n Low 34.1 quantile')
    print(np.percentile(n_arr, 15.87, axis=0))
    low_flux = np.percentile(n_arr, 15.87, axis=0)

    print('\n High 34.1 quantile')
    print(np.percentile(n_arr, 84.13, axis=0))
    high_flux = np.percentile(n_arr, 84.13, axis=0)

    print('\n Standard Error')
    flx_err = [abs(n_cv - low_flux), abs(high_flux - n_cv)]
    flx_err = np.array(flx_err)
    print(flx_err)
    
    median = np.array(median)
    flx_err = np.array(flx_err)
    return median, flx_err

In [ ]:
def get_cov_corr_matrix(cv, universe):

    print(cv.shape)
    print(universe.shape)
    
    #construct covariance matrix
    cov = np.cov(np.subtract(cv[np.newaxis,:], universe).transpose())

    #error = sqrt(diag)
    err = np.sqrt(np.diagonal(cov))

    #construct correlation matrix
    corr = cov/np.outer(err, err)  
    
    return cov, corr

<h3>HNL</h3>

In [ ]:
hnl_flx_universe = []

fig, ax = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#
#Universe
pltdf = df_hnl['mod_t']

for i in range(0, 1000):
    weights = df_hnl['flxw_{}'.format(i)]
    label = ''
    if i == 0:
        label = "Flux Systematic Universes"
        
    hnl_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            alpha = 0.2,
                            linestyle = "-",
                            linewidth = 2,
                            label = label
                        )
    hnl_flx_universe.append(hnl_n)
#-----------------------------------------------------------------#
#Central Value
ax.step(bins, hnl_dict['hnl_cv_plot']
         , color = col_dict['Flamingo']
         , label =  "Central Value"
        )
#-----------------------------------------------------------------#
ax.legend(loc = 'upper right',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(hnl_ymin, hnl_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_flux_weight_universe.png"), dpi=200)
plt.show()

<h3>Median and 68% Quantile</h3>

In [ ]:
hnl_flx_universe = np.array(hnl_flx_universe)
hnl_flx_universe = np.sort(hnl_flx_universe, axis = 0)

hnl_flx_median, hnl_flx_err = get_median_68_quantile(hnl_dict['hnl_cv'], hnl_flx_universe)

#make array for plotting
hnl_flx_median_plot = np.insert(hnl_flx_median, 0, 0)

In [ ]:
hnl_dict['neg_flx_err'] = hnl_flx_err[0]
hnl_dict['pos_flx_err'] = hnl_flx_err[1]

hnl_dict

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, hnl_flx_median_plot, color = col_dict['PastelGreen'], label = "Flux Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, hnl_dict['hnl_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_flx_err'], hnl_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable \nQuantile"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="upper left", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(hnl_ymin, hnl_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_flux_weight_median68.png"), dpi=200)
plt.show()

<h3>Covariance Matrix</h3>

In [ ]:
hnl_flux_cov, _ = get_cov_corr_matrix(hnl_dict['hnl_cv'], hnl_flx_universe)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20, 20))

ax = sns.heatmap(hnl_flux_cov, annot=True, fmt='.2f', cmap = "Spectral")
plt.show()

In [ ]:
hnl_dict['cov_flx_err'] = np.sqrt(np.diag(hnl_flux_cov))

<h3>Comparison</h3>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, hnl_flx_median_plot, color = col_dict['PastelGreen'], label = "Flux Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, hnl_dict['hnl_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_flx_err'], hnl_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable \nQuantile"
           )
#-----------------------------------------------------------------#
#covariance matrix
ax.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['cov_flx_err'], hnl_dict['cov_flx_err']]
            , ls='none'
            , color = col_dict['Coral']
            , capsize=8
            ,label = "Covariance Matrix"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="upper left", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(hnl_ymin, hnl_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_flux_weight_median68.png"), dpi=200)
plt.show()

<h3>Neutrino</h3>

In [ ]:
nu_flx_universe = []

fig, ax = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#
#Universe
pltdf = df_nu['mod_t']

for i in range(0, 1000):
    weights = df_nu['flxw_{}'.format(i)]
    label = ''
    if i == 0:
        label = "Flux Systematic Universes"
        
    nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            alpha = 0.2,
                            linestyle = "-",
                            linewidth = 2,
                            label = label
                        )
    nu_flx_universe.append(nu_n)
#-----------------------------------------------------------------#
#Central Value
ax.step(bins, nu_dict['nu_cv_plot']
         , color = col_dict['Flamingo']
         , label =  "Central Value"
        )
#-----------------------------------------------------------------#
ax.legend(loc = 'upper right',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_flux_weight_universe.png"), dpi=200)
plt.show()

In [ ]:
nu_flx_universe = np.array(nu_flx_universe)
nu_flx_universe = np.sort(nu_flx_universe, axis = 0)

nu_flx_median, nu_flx_err = get_median_68_quantile(nu_dict['nu_cv'], nu_flx_universe)

#make array for plotting
nu_flx_median_plot = np.insert(nu_flx_median, 0, 0)

In [ ]:
nu_dict['neg_flx_err'] = nu_flx_err[0]
nu_dict['pos_flx_err'] = nu_flx_err[1]

nu_dict

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_flx_median_plot, color = col_dict['PastelGreen'], label = "Flux Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_flx_err'] ,nu_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="upper left", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_flux_weight_median68.png"), dpi=200)
plt.show()

<h3>Covariance Matrix </h3>

In [ ]:
nu_flux_cov, _ = get_cov_corr_matrix(nu_dict['nu_cv'], nu_flx_universe)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20, 20))

ax = sns.heatmap(nu_flux_cov, annot=True, fmt='.2f', cmap = "Spectral")
plt.show()

In [ ]:
nu_dict['cov_flx_err'] = np.sqrt(np.diag(nu_flux_cov))

<h3>Comparison</h3>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_flx_median_plot, color = col_dict['PastelGreen'], label = "Flux Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_flx_err'] ,nu_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#covariance matrix
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['cov_flx_err'] ,nu_dict['cov_flx_err']]
            , ls='none'
            , color = col_dict['Coral']
            , capsize=8
            ,label = "Covariance Matrix"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="upper left", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_flux_weight_median68.png"), dpi=200)
plt.show()

<h1>Cross Section Weights</h1>

<h3>Uni Sim</h3>

In [ ]:
nu_unisim_arr = []

fig, ax = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#
#Universe
pltdf = df_nu['mod_t']
weights = df_nu['xsec_nom']
    
for name in unisim_list:
    weights = df_nu[name]
    
    nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            alpha = 0.2,
                            linestyle = "-",
                            linewidth = 2,
                            label = name
                        )
    nu_unisim_arr.append(nu_n)
    
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
ax.legend(loc = 'upper right',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_unisim_xsec_weight_universe.png"), dpi=200)
plt.show()

In [ ]:
#Should be 4 unisim
nu_unisim_arr = np.array(nu_unisim_arr)
nu_unisim_err = nu_unisim_arr - nu_dict['nu_cv']
nu_unisim_err

In [ ]:
#TODO: Unisim - IGNORE FOR NOW
#nu_err['neg_flx_err'] = nu_flx_err[0]
#nu_err

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

#-----------------------------------------------------------------#

#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#TODO: CURRENTLY PLOTTING 
#universe 1 sigma
for i in range(0, nu_unisim_err.shape[0]):
    label = ""
    if i == 0:
        label = '4 UniSim Variables'
    ax.errorbar(bins_mid, nu_dict['nu_cv']
                , yerr=(np.maximum(0, - nu_unisim_err[i]), np.maximum(0, nu_unisim_err[i]))
                , ls='none'
                , color = col_dict['Mauve']
                , capsize=8
                , label = label
                )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_weight_unisim_1direction.png"), dpi=200)
plt.show()

<h3>Multi Sigma - Take 1 Sigma Directly</h3>

In [ ]:
nu_multisigma_pos1_arr = []
nu_multisigma_neg1_arr = []
    
for name in multisigma_list:
    
    #check if weight contains anything other than 1
    pos1_reweight = all(p == 1 for p in df_nu[name+"_pos1"])
    neg1_reweight = all(p == 1 for p in df_nu[name+"_neg1"])

    
    if ( (pos1_reweight == False) or (neg1_reweight == False)):

        fig, ax = plt.subplots(1,1, figsize = (6,4))

        xmin, xmax, xnbin = xmin, xmax, xnbin
        xlimmin, xlimmax = xmin, xmax
        #-----------------------------------------------------------------#
        pltdf = df_nu['mod_t']
        
        #-----------------------------------------------------------------#
        weights = df_nu[name+"_pos1"]
        label = name.split("_")[3]
    
        nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            #alpha = 0.2,
                            linestyle = "-",
                            linewidth = 2,
                            label = label +" +1 sigma"
                        )
        nu_multisigma_pos1_arr.append(nu_n)
        #-----------------------------------------------------------------#
        #for name in multisigma_list:
        weights = df_nu[name+"_neg1"]
        label = name.split("_")[3]
    
        nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["Purple"],
                            #alpha = 0.2,
                            linestyle = "-",
                            linewidth = 2,
                            label = label +" -1 sigma"
                        )
        nu_multisigma_neg1_arr.append(nu_n)
        #-----------------------------------------------------------------#
        #central value
        ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")

        #-----------------------------------------------------------------#

        ax.legend(loc = 'best',fontsize = 14)
        #ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol =2)

        plot_tick(ax, 16)
        plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(nu_ymin, nu_ymax)
        #-----------------------------------------------------------------#
        fig.tight_layout()

        if ifSave:
            plt.savefig(savePath+str("nu_beam_bucket_multisigma_xsec_weight_universe"+name+".png"), dpi=200)
        plt.show()

In [ ]:
nu_multisigma_neg1_sort_arr = []
nu_multisigma_pos1_sort_arr = []

for neg,pos in zip(nu_multisigma_neg1_arr, nu_multisigma_pos1_arr):

    neg_arr = []
    pos_arr = []
    
    for negItem, posItem in zip(neg, pos):
        neg_arr.append(min(negItem, posItem))
        pos_arr.append(max(negItem, posItem))
        
    nu_multisigma_neg1_sort_arr.append(neg_arr)
    nu_multisigma_pos1_sort_arr.append(pos_arr)

#print(nu_multisigma_neg1_sort_arr)
for neg,pos in zip(nu_multisigma_neg1_sort_arr, nu_multisigma_pos1_sort_arr):
    print()
    print(neg)
    print(pos)

In [ ]:
#Negative
nu_multisigma_neg1_err = abs(nu_dict['nu_cv'] - nu_multisigma_neg1_sort_arr)

#Positve
nu_multisigma_pos1_err = abs(nu_dict['nu_cv'] - nu_multisigma_pos1_sort_arr)

In [ ]:
nu_dict['neg_xsec_multisigma'] = nu_multisigma_neg1_err
nu_dict['pos_xsec_multisigma'] = nu_multisigma_pos1_err

for i in range(0, 4):
    print(i)
    print(nu_dict['neg_xsec_multisigma'][i])
    print(nu_dict['pos_xsec_multisigma'][i])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))

#-----------------------------------------------------------------#

#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#

#universe 1 sigma
for i in range(0, nu_multisigma_neg1_err.shape[0]):
    label = ""
    if i == 0:
        label = '24 MultiSigma Variables'
    ax.errorbar(bins_mid
                , nu_dict['nu_cv']
                , [nu_dict['neg_xsec_multisigma'][i],nu_dict['pos_xsec_multisigma'][i]]
                , ls='none'
                , color = col_dict['Mauve']
                , capsize=8
                , label = label
               )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()
if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_weight_multisigma_1sigband.png"), dpi=200)
plt.show()

<h3>Multi Sigma - TODO: Throw Universe Using TSpline and Construct Covariance Matrix</h3>

<h3>Multi Sim</h3>

In [ ]:
nu_xsec_multisim_universe = []

fig, ax = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#
#Universe
pltdf = df_nu['mod_t']

for i in range(0, 500):
    weights = df_nu['xsecw_{}'.format(i)]
    label = ''
    if i == 0:
        label = "XSec MultiSim \nSystematic Universes"
        
    nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            linestyle = "-",
                            linewidth = 2,
                            label = label
                        )
    nu_xsec_multisim_universe.append(nu_n)
    
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")

#-----------------------------------------------------------------#
ax.legend(loc = 'upper right',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_multisim_weight_universe.png"), dpi=200)
plt.show()

In [ ]:
nu_xsec_multisim_universe = np.array(nu_xsec_multisim_universe)
nu_xsec_multisim_universe = np.sort(nu_xsec_multisim_universe, axis = 0)

nu_xsec_multisim_median, nu_xsec_multisim_err = get_median_68_quantile(nu_dict['nu_cv'], nu_xsec_multisim_universe)

#make array for plotting
nu_xsec_multisim_median_plot = np.insert(nu_xsec_multisim_median, 0, 0)

In [ ]:
nu_dict['neg_xsec_multisim'] = nu_xsec_multisim_err[0]
nu_dict['pos_xsec_multisim'] = nu_xsec_multisim_err[1]
nu_dict

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_xsec_multisim_median_plot, color = col_dict['PastelGreen'], label = "XSec MultiSim \nSystematic Universes Median \n7 Correlated Variables")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_xsec_multisim_err
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_multisim_weight_median68.png"), dpi=200)
plt.show()

<h3>Covarariance Matrix</h3>

In [ ]:
nu_xsec_multisim_cov, _ = get_cov_corr_matrix(nu_dict['nu_cv'], nu_xsec_multisim_universe)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20, 20))

ax = sns.heatmap(nu_xsec_multisim_cov, annot=True, fmt='.2f', cmap = "Spectral")
plt.show()

In [ ]:
nu_dict['cov_xsec_multisim_err'] = np.sqrt(np.diag(nu_xsec_multisim_cov))

<h3>Comparison</h3>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_xsec_multisim_median_plot, color = col_dict['PastelGreen'], label = "XSec MultiSim \nSystematic Universes Median \n7 Correlated Variables")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_xsec_multisim_err
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_dict['cov_xsec_multisim_err']
            , ls='none'
            , color = col_dict['Coral']
            , capsize=8
            ,label = "Covariance Matrix"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_multisim_weight_median68.png"), dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_xsec_multisim_median_plot, color = col_dict['PastelGreen'], label = "XSec MultiSim \nSystematic Universes Median \n7 Correlated Variables")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_xsec_multisim_err
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_dict['cov_xsec_multisim_err']
            , ls='none'
            , color = col_dict['Coral']
            , capsize=8
            ,label = "Covariance Matrix"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

#ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, 25)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_multisim_weight_median68.png"), dpi=200)
plt.show()

<h3> Quadrature Sum All Xsec Systematics </h3>

In [ ]:
# function to calculate the sum of an array
def quadratureSumArray(arr):
    quadSum = 0
    
    for element in arr: 
        quadSum = quadSum + element**2
        
    return np.sqrt(quadSum)

In [ ]:
#Negative
nu_dict['neg_xsec_multisim'] = nu_dict['neg_xsec_multisim'].reshape(1,19)

nu_xsec_neg = np.append(nu_dict['neg_xsec_multisim'], nu_dict['neg_xsec_multisigma'], axis=0)
print(nu_xsec_neg.shape)
#print(nu_xsec_neg)

nu_xsec_neg_quadSum = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_xsec_neg)

print(nu_xsec_neg_quadSum.shape)
print(nu_xsec_neg_quadSum)

nu_dict['neg_xsec_quadSum'] = nu_xsec_neg_quadSum


In [ ]:
#Positive
nu_dict['pos_xsec_multisim'] = nu_dict['pos_xsec_multisim'].reshape(1,19)

nu_xsec_pos = np.append(nu_dict['pos_xsec_multisim'], nu_dict['pos_xsec_multisigma'], axis=0)
print(nu_xsec_pos.shape)
#print(nu_xsec_neg)

nu_xsec_pos_quadSum = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_xsec_pos)

print(nu_xsec_pos_quadSum.shape)
print(nu_xsec_pos_quadSum)

nu_dict['pos_xsec_quadSum'] = nu_xsec_pos_quadSum


In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_xsec_quadSum'], nu_dict['pos_xsec_quadSum']],
            ls='none',
            color = col_dict['Mauve'],
            capsize=8
            ,label = "Combined XSec Systematic"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (1x10$^{21}$ POT)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_weight_combined.png"), dpi=200)
plt.show()

In [ ]:
fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2, figsize = (12,8))#, sharex = True, sharey = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#========================================================================#
#UNISIM
#central value
ax1.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#TODO: CURRENTLY PLOTTING 
#universe 1 sigma
for i in range(0, nu_unisim_err.shape[0]):
    label = ""
    if i == 0:
        label = '4 UniSim Variables'
    ax1.errorbar(bins_mid, nu_dict['nu_cv']
                , yerr=(np.maximum(0, - nu_unisim_err[i]), np.maximum(0, nu_unisim_err[i]))
                , ls='none'
                , color = col_dict['Mauve']
                , capsize=8
                , label = label
                )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax1,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax1, 16)

ax1.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(nu_ymin, nu_ymax)

#========================================================================#
#MULTISIGMA
#central value
ax2.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#

#universe 1 sigma
for i in range(0, nu_multisigma_neg1_err.shape[0]):
    label = ""
    if i == 0:
        label = '24 MultiSigma Variables'
    ax2.errorbar(bins_mid
                , nu_dict['nu_cv']
                , [nu_dict['neg_xsec_multisigma'][i],nu_dict['pos_xsec_multisigma'][i]]
                , ls='none'
                , color = col_dict['Mauve']
                , capsize=8
                , label = label
               )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax2, 16)

ax2.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(nu_ymin, nu_ymax)

#========================================================================#
#MULTISIM
#median
ax3.step(bins, nu_xsec_multisim_median_plot, color = col_dict['PastelGreen'], label = "XSec MultiSim \nSystematic Universes Median \n7 Correlated Variables")
#-----------------------------------------------------------------#
#central value
ax3.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax3.errorbar(bins_mid, nu_dict['nu_cv']
            , nu_xsec_multisim_err
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            ,label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax3,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax3, 16)

ax3.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax3.set_xlim(xmin, xmax)
ax3.set_ylim(nu_ymin, nu_ymax)

#========================================================================#
#COMBINED
#central value
ax4.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax4.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_xsec_quadSum'], nu_dict['pos_xsec_quadSum']],
            ls='none',
            color = col_dict['Mauve'],
            capsize=8
            ,label = "Combined XSec Systematic"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax4,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax4, 16)

ax4.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax4.set_xlim(xmin, xmax)
ax4.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_xsec_weight_all.png"), dpi=200)
plt.show()

<h1>Geant4 ReInteractions Weights</h1>

In [ ]:
nu_g4_multisim_universe = []

fig, ax = plt.subplots(1,1, figsize = (6,4))

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#-----------------------------------------------------------------#
pltdf = df_nu['mod_t']

for i in range(0, 1000):
    weights = df_nu['g4w_{}'.format(i)]
    label = ''
    if i == 0:
        label = "Geant4 Systematic Universes"
        
    nu_n, _, _ = ax.hist(
                            pltdf,
                            bins = np.arange(xmin, xmax+(xmax-xmin)/xnbin, (xmax-xmin)/xnbin),
                            weights = weights,
                            density = False,
                            histtype="step",
                            edgecolor = col_dict["PastelGreen"],
                            linestyle = "-",
                            linewidth = 2,
                            label = label
                        )
    nu_g4_multisim_universe.append(nu_n)
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
ax.legend(loc = 'upper right',fontsize = 14)

plot_tick(ax, 16)
plot_title(ax, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_g4_multisim_weight_universe.png"), dpi=200)
plt.show()

In [ ]:
nu_g4_multisim_universe = np.array(nu_g4_multisim_universe)
nu_g4_multisim_universe = np.sort(nu_g4_multisim_universe, axis = 0)

nu_g4_multisim_median, nu_g4_multisim_err = get_median_68_quantile(nu_dict['nu_cv'], nu_g4_multisim_universe)

#make array for plotting
nu_g4_multisim_median_plot = np.insert(nu_g4_multisim_median, 0, 0)

In [ ]:
nu_dict['neg_g4_err'] = nu_g4_multisim_err[0]
nu_dict['pos_g4_err'] = nu_g4_multisim_err[1]
nu_dict.keys()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_g4_multisim_median_plot , color = col_dict['PastelGreen'], label = "Geant4 Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_g4_err'], nu_dict['pos_g4_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            , label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_g4_multisim_weight_median68.png"), dpi=200)
plt.show()

<h3>Covariance Matrix</h3>

In [ ]:
nu_g4_multisim_cov, _ = get_cov_corr_matrix(nu_dict['nu_cv'], nu_g4_multisim_universe)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20, 20))

ax = sns.heatmap(nu_g4_multisim_cov, annot=True, fmt='.2f', cmap = "Spectral")
plt.show()

In [ ]:
nu_dict['cov_g4_multisim_err'] = np.sqrt(np.diag(nu_g4_multisim_cov))

<h3>Comparison</h3>

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
#-----------------------------------------------------------------#
#median
ax.step(bins, nu_g4_multisim_median_plot , color = col_dict['PastelGreen'], label = "Geant4 Systematic \nUniverses Median")
#-----------------------------------------------------------------#
#central value
ax.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = "Central Value")
#-----------------------------------------------------------------#
#universe 1 sigma
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_g4_err'], nu_dict['pos_g4_err']]
            , ls='none'
            , color = col_dict['Mauve']
            , capsize=8
            , label = "68% Most Probable Quantile"
           )
#-----------------------------------------------------------------#
#covariance matrix
ax.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['cov_g4_multisim_err'], nu_dict['cov_g4_multisim_err']]
            , ls='none'
            , color = col_dict['Coral']
            , capsize=8
            , label = "Covariance Matrix"
           )
#-----------------------------------------------------------------#
#tick stuff
plot_title(ax,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Slices (No Scaling)", 16)
plot_tick(ax, 16)

ax.legend(loc="best", fontsize=14, fancybox=True, ncol = 1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(nu_ymin, nu_ymax)
#-----------------------------------------------------------------#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_g4_multisim_weight_median68.png"), dpi=200)
plt.show()

<h1>Plot Everything</h1>

<h2>HNL - Directly Sum 1 Sigma </h2>

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12,4))#, sharex = True, sharey = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#
#STATS
ax1.step(bins, hnl_dict['hnl_cv_plot'], color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU))
        )

ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_stat_err'],hnl_dict['pos_stat_err']]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            , label = "Statistics"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax)

#=========================================================================#
#FLUX
#-----------------------------------------------------------------#
#central value
ax2.step(bins, hnl_dict['hnl_cv_plot'], color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU)))
#-----------------------------------------------------------------#
#universe 1 sigma
ax2.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_flx_err'],hnl_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            ,label = "Flux"
           )

#-----------------------------------------------------------------#
ax2.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(hnl_ymin, hnl_ymax)

#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_standard_error.png"), dpi=200)
    
plt.show()

In [ ]:
hnl_dict['neg_stat_err'] = hnl_dict['neg_stat_err'].reshape(1,19)
hnl_dict['neg_flx_err'] = hnl_dict['neg_flx_err'].reshape(1,19)

hnl_neg_err = np.append(hnl_dict['neg_stat_err']
                        , hnl_dict['neg_flx_err']
                        , axis=0)

hnl_neg_quad_err = np.apply_along_axis(quadratureSumArray, axis=0, arr=hnl_neg_err)

hnl_dict['neg_quadSum_err'] = hnl_neg_quad_err
#------------------------------------------------------------------#

hnl_dict['pos_flx_err'] = hnl_dict['pos_flx_err'].reshape(1,19)
hnl_dict['pos_stat_err'] = hnl_dict['pos_stat_err'].reshape(1,19)

hnl_pos_err = np.append(hnl_dict['pos_stat_err']
                        , hnl_dict['pos_flx_err']
                        , axis=0)

hnl_pos_quad_err = np.apply_along_axis(quadratureSumArray, axis=0, arr=hnl_pos_err)

hnl_dict['pos_quadSum_err'] = hnl_pos_quad_err
#------------------------------------------------------------------#
print("\n Negative Direction")
print(hnl_neg_err)
print(hnl_neg_quad_err)

#------------------------------------------------------------------#
print("\n Positive Direction")
print(hnl_pos_err)
print(hnl_pos_quad_err)

In [ ]:
#FOR PLOTTING

#Stat
hnl_stat_neg_diff = hnl_dict['neg_stat_err'] / hnl_dict['hnl_cv']
hnl_stat_neg_diff = np.insert(hnl_stat_neg_diff, 0, 0)

hnl_stat_pos_diff = hnl_dict['pos_stat_err'] / hnl_dict['hnl_cv']
hnl_stat_pos_diff = np.insert(hnl_stat_pos_diff, 0, 0)

#Flux
hnl_flx_neg_diff = hnl_dict['neg_flx_err'] / hnl_dict['hnl_cv']
hnl_flx_neg_diff = np.insert(hnl_flx_neg_diff, 0, 0)

hnl_flx_pos_diff = hnl_dict['pos_flx_err'] / hnl_dict['hnl_cv']
hnl_flx_pos_diff = np.insert(hnl_flx_pos_diff, 0, 0)

#Quad Sum
hnl_quadsum_neg_diff = hnl_dict['neg_quadSum_err'] / hnl_dict['hnl_cv']
hnl_quadsum_neg_diff = np.insert(hnl_quadsum_neg_diff, 0, 0)

hnl_quadsum_pos_diff = hnl_dict['pos_quadSum_err'] / hnl_dict['hnl_cv']
hnl_quadsum_pos_diff = np.insert(hnl_quadsum_pos_diff, 0, 0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#

ax1.step(bins, hnl_dict['hnl_cv_plot'], color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU))
        )
#-----------------------------------------------------------------#
#QuadSum
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_quadSum_err'], hnl_dict['pos_quadSum_err']]
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            ,label = "Quadrature Sum"
           )
#-----------------------------------------------------------------#
#STATS
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_stat_err'].flatten(), hnl_dict['pos_stat_err'].flatten()]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#
#Flux
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , [hnl_dict['neg_flx_err'].flatten(), hnl_dict['pos_flx_err'].flatten()]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            ,label = "Flux"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", '',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax)
#=========================================================================#
#-----------------------------------------------------------------#
#QuadSum
ax2.step(bins, hnl_quadsum_neg_diff  * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, hnl_quadsum_pos_diff
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, hnl_stat_neg_diff  * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, hnl_stat_pos_diff
         , color = col_dict['Lavender']
        )
#-----------------------------------------------------------------#
#Flux
ax2.step(bins, hnl_flx_neg_diff  * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, hnl_flx_pos_diff
         , color = col_dict['PastelGreen']
        )

#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.2, 0.2)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_combined_standard_error.png"), dpi=200)
    
plt.show()

<h3>Scale To POT AND Umu Coupling</h3>

In [ ]:
scale_factor = df_hnl['scale_pot'].unique()[0]
fitU = plotU
print(scale_factor)
print(fitU)


scale = 1/80
scale_factor = scale_factor * scale
fitU = getUfromScaleFactor(fitU, scale)
print(scale_factor)
print(fitU)


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#

ax1.step(bins, hnl_dict['hnl_cv_plot']* scale_factor
         , color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(fitU))
        )
#-----------------------------------------------------------------#
#QuadSum
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , [hnl_quadsum_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
               , hnl_quadsum_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor]
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            ,label = "Quadrature Sum"
           )
#-----------------------------------------------------------------#
#STATS
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , [hnl_stat_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
               , hnl_stat_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#
#Flux
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , [hnl_flx_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
               , hnl_flx_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            ,label = "Flux"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", '',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax * scale_factor)
#=========================================================================#
#-----------------------------------------------------------------#
#QuadSum
ax2.step(bins, hnl_quadsum_neg_diff * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, hnl_quadsum_pos_diff
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, hnl_stat_neg_diff   * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, hnl_stat_pos_diff
         , color = col_dict['Lavender']
        )
#-----------------------------------------------------------------#
#Flux
ax2.step(bins, hnl_flx_neg_diff * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, hnl_flx_pos_diff
         , color = col_dict['PastelGreen']
        )

#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.2, 0.2)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_combined_standard_error_scaleToPOT.png"), dpi=200)
    
plt.show()

In [ ]:
hnl_dict['hnl_cv_scale'] = hnl_dict['hnl_cv'] * scale_factor

#stat
hnl_dict['neg_stat_err_scale'] = hnl_stat_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
hnl_dict['pos_stat_err_scale'] = hnl_stat_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor

#flux
hnl_dict['neg_flx_err_scale'] = hnl_flx_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
hnl_dict['pos_flx_err_scale'] = hnl_flx_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor

#quadSum
hnl_dict['neg_quadSum_err_scale'] = hnl_quadsum_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor
hnl_dict['pos_quadSum_err_scale'] = hnl_quadsum_pos_diff[1:] * hnl_dict['hnl_cv'] * scale_factor

<h2>HNL - Covariance Matrix Way </h2>

In [ ]:
hnl_cv = hnl_dict['hnl_cv']

#stat
hnl_stat_cov = np.diag(hnl_cv) #[some NxN covariance matrix e.g. np.diag(cv) for statistical]
hnl_stat_err = np.sqrt(np.diag(hnl_stat_cov))


hnl_stat_frac = hnl_stat_cov / np.outer(hnl_cv, hnl_cv) # N / (N * N)
hnl_stat_frac_err = np.sqrt(np.diag(hnl_stat_frac)) # sqrt(N)


#flux
hnl_flx_cov, _ = get_cov_corr_matrix(hnl_cv, hnl_flx_universe)
hnl_flx_err = np.sqrt(np.diag(hnl_flx_cov))

hnl_flx_frac = hnl_flx_cov / np.outer(hnl_cv, hnl_cv)
hnl_flx_frac_err = np.sqrt(np.diag(hnl_flx_frac)) # sqrt(N)

#add covariance matrix
hnl_combined_cov = hnl_stat_cov + hnl_flx_cov
hnl_combined_err = np.sqrt(np.diag(hnl_combined_cov))

hnl_combined_frac = hnl_combined_cov / np.outer(hnl_cv, hnl_cv) # N / (N * N)
hnl_combined_frac_err = np.sqrt(np.diag(hnl_combined_frac)) # sqrt(N)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#

ax1.step(bins, hnl_dict['hnl_cv_plot']
         , color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(simU))
        )
#-----------------------------------------------------------------#
#Quad Sum
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , hnl_combined_err
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            ,label = "Quadrature Sum"
           )
#-----------------------------------------------------------------#
#STATS
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , hnl_stat_err
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#
#FLUX
ax1.errorbar(bins_mid, hnl_dict['hnl_cv']
            , hnl_flx_err
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            ,label = "Flux"
           )


#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", '',  r"Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax)
#=========================================================================#
#-----------------------------------------------------------------#
#QUAD SUM
ax2.step(bins, np.insert(hnl_combined_frac_err, 0,0) * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, np.insert(hnl_combined_frac_err, 0,0) 
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, np.insert(hnl_stat_frac_err, 0,0) * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, np.insert(hnl_stat_frac_err, 0,0) 
         , color = col_dict['Lavender']
        )
#-----------------------------------------------------------------#
#FLUX
ax2.step(bins, np.insert(hnl_flx_frac_err, 0,0) * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, np.insert(hnl_flx_frac_err, 0,0) 
         , color = col_dict['PastelGreen']
        )
#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.2, 0.2)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_combined_covariance.png"), dpi=200)
    
plt.show()

<h3>Scale To POT AND Umu Coupling</h3>

In [ ]:
hnl_cv_scale = hnl_cv * scale_factor

#stat
hnl_stat_cov_scale = np.outer(hnl_cv_scale, hnl_cv_scale) * hnl_stat_frac
hnl_stat_err_scale = np.sqrt(np.diag(hnl_stat_cov_scale))

hnl_stat_frac_scale = hnl_stat_cov_scale / np.outer(hnl_cv_scale, hnl_cv_scale) # N / (N * N)
hnl_stat_frac_err_scale = np.sqrt(np.diag(hnl_stat_frac_scale)) # sqrt(N)


#flux
hnl_flx_cov_scale = np.outer(hnl_cv_scale, hnl_cv_scale) * hnl_flx_frac
hnl_flx_err_scale = np.sqrt(np.diag(hnl_flx_cov_scale))

hnl_flx_frac_scale = hnl_flx_cov_scale / np.outer(hnl_cv_scale, hnl_cv_scale)
hnl_flx_frac_err_scale = np.sqrt(np.diag(hnl_flx_frac_scale)) # sqrt(N)


#add covariance matrix
hnl_combined_cov_scale = np.outer(hnl_cv_scale, hnl_cv_scale) * hnl_combined_frac
hnl_combined_err_scale = np.sqrt(np.diag(hnl_combined_cov_scale))

hnl_combined_frac_scale = hnl_combined_cov_scale / np.outer(hnl_cv_scale, hnl_cv_scale) # N / (N * N)
hnl_combined_frac_err_scale = np.sqrt(np.diag(hnl_combined_frac_scale)) # sqrt(N)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#

ax1.step(bins, hnl_dict['hnl_cv_plot'] * scale_factor
         , color = col_dict['Flamingo']
         , label =  "HNL Central Value \nM = 200 MeV\n|U$_{\mu4}|^{2} = $" +str("{:.3g}".format(fitU))
        )
#-----------------------------------------------------------------#
#Quad Sum
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , hnl_combined_err_scale
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            ,label = "Quadrature Sum"
           )
#-----------------------------------------------------------------#
#STATS
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , hnl_stat_err_scale
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            ,label = "Statistics"
           )
#-----------------------------------------------------------------#
#FLUX
ax1.errorbar(bins_mid, hnl_dict['hnl_cv'] * scale_factor
            , hnl_flx_err_scale
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            ,label = "Flux"
           )


#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", '',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hnl_ymin, hnl_ymax * scale_factor)
#=========================================================================#
#-----------------------------------------------------------------#
#QUAD SUM
ax2.step(bins, np.insert(hnl_combined_frac_err_scale, 0,0) * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, np.insert(hnl_combined_frac_err_scale, 0,0) 
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, np.insert(hnl_stat_frac_err_scale, 0,0) * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, np.insert(hnl_stat_frac_err_scale, 0,0) 
         , color = col_dict['Lavender']
        )
#-----------------------------------------------------------------#
#FLUX
ax2.step(bins, np.insert(hnl_flx_frac_err_scale, 0,0) * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, np.insert(hnl_flx_frac_err_scale, 0,0) 
         , color = col_dict['PastelGreen']
        )
#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-0.2, 0.2)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("hnl_beam_bucket_combined_covariance_scaleToPOT.png"), dpi=200)
    
plt.show()

<h2>Neutrino - Directly Sum 1 Sigma </h2>

In [ ]:
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2, figsize = (12,8))#, sharey = True, sharex= True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax
#=========================================================================#
#STATS
#-----------------------------------------------------------------#
#central value
ax1.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = r" $\nu$ Central Value")

ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_stat_err'],nu_dict['pos_stat_err']]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            , label = "Statistics"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(nu_ymin, nu_ymax)

#=========================================================================#
#FLUX
#-----------------------------------------------------------------#
#central value
ax2.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = r" $\nu$ Central Value")

#-----------------------------------------------------------------#
#universe 1 sigma
ax2.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_flx_err'],nu_dict['pos_flx_err']]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            , label = "Flux"
           )

#-----------------------------------------------------------------#
ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(nu_ymin, nu_ymax)

#=========================================================================#
#XSEC
#-----------------------------------------------------------------#
#central value
ax3.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = r" $\nu$ Central Value")

#-----------------------------------------------------------------#
#universe 1 sigma
ax3.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_xsec_quadSum'],nu_dict['pos_xsec_quadSum']]
            , ls='none'
            , color = col_dict['Peach']
            , capsize=8
            , label = r" $\nu$ XSec"
           )

#-----------------------------------------------------------------#
ax3.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax3, 16)
plot_title(ax3, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax3.set_xlim(xmin, xmax)
ax3.set_ylim(nu_ymin, nu_ymax)
#=========================================================================#
#G4
#-----------------------------------------------------------------#
#central value
ax4.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = r" $\nu$ Central Value")

#-----------------------------------------------------------------#
#universe 1 sigma
ax4.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_g4_err'],nu_dict['pos_g4_err']]
            , ls='none'
            , color = col_dict['SkyBlue']
            , capsize=8
            , label = "Geant4 Re-Interactions"
           )

#-----------------------------------------------------------------#
ax4.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax4, 16)
plot_title(ax4, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax4.set_xlim(xmin, xmax)
ax4.set_ylim(nu_ymin, nu_ymax)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_standard_error.png"), dpi=200)
    
plt.show()

In [ ]:
nu_dict['neg_stat_err'] = nu_dict['neg_stat_err'].reshape(1,19)
nu_dict['neg_flx_err'] = nu_dict['neg_flx_err'].reshape(1,19)
nu_dict['neg_xsec_quadSum'] = nu_dict['neg_xsec_quadSum'].reshape(1,19)
nu_dict['neg_g4_err'] = nu_dict['neg_g4_err'].reshape(1,19)

nu_neg_err = np.append(nu_dict['neg_stat_err'], nu_dict['neg_flx_err'], axis=0)
nu_neg_err = np.append(nu_neg_err, nu_dict['neg_xsec_quadSum'], axis=0)
nu_neg_err = np.append(nu_neg_err, nu_dict['neg_g4_err'], axis=0)

print("\n Negative Direction")
print(nu_neg_err)

#------------------------------------------------------------------#

nu_dict['pos_stat_err'] = nu_dict['pos_stat_err'].reshape(1,19)
nu_dict['pos_flx_err'] = nu_dict['pos_flx_err'].reshape(1,19)
nu_dict['pos_xsec_quadSum'] = nu_dict['pos_xsec_quadSum'].reshape(1,19)
nu_dict['pos_g4_err'] = nu_dict['pos_g4_err'].reshape(1,19)

nu_pos_err = np.append(nu_dict['pos_stat_err'], nu_dict['pos_flx_err'], axis=0)
nu_pos_err = np.append(nu_pos_err, nu_dict['pos_xsec_quadSum'], axis=0)
nu_pos_err = np.append(nu_pos_err, nu_dict['pos_g4_err'], axis=0)

print("\n Positive Direction")
print(nu_pos_err)

#------------------------------------------------------------------#

nu_neg_quad_err = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_neg_err)

nu_dict['neg_quadSum_err'] = nu_neg_quad_err
#------------------------------------------------------------------#

nu_pos_quad_err = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_pos_err)

nu_dict['pos_quadSum_err'] = nu_pos_quad_err

In [ ]:
nu_dict.keys()
print(nu_dict['neg_quadSum_err'])
print(nu_dict['pos_quadSum_err'])

In [ ]:
#FOR PLOTTING

#Stat
nu_stat_neg_diff = nu_dict['neg_stat_err'] / nu_dict['nu_cv']
nu_stat_neg_diff = np.insert(nu_stat_neg_diff, 0, 0)
nu_stat_neg_diff[np.isnan(nu_stat_neg_diff)] = 0

nu_stat_pos_diff = nu_dict['pos_stat_err'] / nu_dict['nu_cv']
nu_stat_pos_diff = np.insert(nu_stat_pos_diff, 0, 0)
nu_stat_pos_diff[np.isnan(nu_stat_pos_diff)] = 0

#Flux
nu_flx_neg_diff = nu_dict['neg_flx_err'] / nu_dict['nu_cv']
nu_flx_neg_diff = np.insert(nu_flx_neg_diff, 0, 0)
nu_flx_neg_diff[np.isnan(nu_flx_neg_diff)] = 0

nu_flx_pos_diff = nu_dict['pos_flx_err'] / nu_dict['nu_cv']
nu_flx_pos_diff = np.insert(nu_flx_pos_diff, 0, 0)
nu_flx_pos_diff[np.isnan(nu_flx_pos_diff)] = 0

#Quad Sum
nu_quadsum_neg_diff = nu_dict['neg_quadSum_err'] / nu_dict['nu_cv']
nu_quadsum_neg_diff = np.insert(nu_quadsum_neg_diff, 0, 0)
nu_quadsum_neg_diff[np.isnan(nu_quadsum_neg_diff)] = 0

nu_quadsum_pos_diff = nu_dict['pos_quadSum_err'] / nu_dict['nu_cv']
nu_quadsum_pos_diff = np.insert(nu_quadsum_pos_diff, 0, 0)
nu_quadsum_pos_diff[np.isnan(nu_quadsum_pos_diff)] = 0

#XSec
nu_xsec_neg_diff = nu_dict['neg_xsec_quadSum'] / nu_dict['nu_cv']
nu_xsec_neg_diff = np.insert(nu_xsec_neg_diff, 0, 0)
nu_xsec_neg_diff[np.isnan(nu_xsec_neg_diff)] = 0

nu_xsec_pos_diff = nu_dict['pos_xsec_quadSum'] / nu_dict['nu_cv']
nu_xsec_pos_diff = np.insert(nu_xsec_pos_diff, 0, 0)
nu_xsec_pos_diff[np.isnan(nu_xsec_pos_diff)] = 0

#G4
nu_g4_neg_diff = nu_dict['neg_g4_err'] / nu_dict['nu_cv']
nu_g4_neg_diff = np.insert(nu_g4_neg_diff, 0, 0)
nu_g4_neg_diff[np.isnan(nu_g4_neg_diff)] = 0

nu_g4_pos_diff = nu_dict['pos_g4_err'] / nu_dict['nu_cv']
nu_g4_pos_diff = np.insert(nu_g4_pos_diff, 0, 0)
nu_g4_pos_diff[np.isnan(nu_g4_pos_diff)] = 0

In [ ]:
print(nu_dict['neg_stat_err'] / nu_dict['nu_cv'])
print(nu_dict['pos_stat_err'] / nu_dict['nu_cv'])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

#-----------------------------------------------------------------#
#central value
ax1.step(bins, nu_dict['nu_cv_plot'], color = col_dict['Flamingo'], label = r" $\nu$ Central Value")
#=========================================================================#
#QuadSum
ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_quadSum_err'],nu_dict['pos_quadSum_err']]
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            , label = "Quadrature Sum"
           )

#=========================================================================#
#XSEC
ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_xsec_quadSum'].flatten(),nu_dict['pos_xsec_quadSum'].flatten()]
            , ls='none'
            , color = col_dict['Peach']
            , capsize=8
            , label = r" $\nu$ XSec"
           )

#=========================================================================#
#FLUX
ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_flx_err'].flatten(),nu_dict['pos_flx_err'].flatten()]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            , label = "Flux"
           )

#=========================================================================#
#STATS
ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_stat_err'].flatten(),nu_dict['pos_stat_err'].flatten()]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            , label = "Statistics"
           )
#=========================================================================#
#G4
ax1.errorbar(bins_mid, nu_dict['nu_cv']
            , [nu_dict['neg_g4_err'].flatten(),nu_dict['pos_g4_err'].flatten()]
            , ls='none'
            , color = col_dict['SkyBlue']
            , capsize=8
            , label = "Geant4 \nRe-Interactions"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(nu_ymin, nu_ymax)
#=========================================================================#
#-----------------------------------------------------------------#
#QuadSum
ax2.step(bins, nu_quadsum_neg_diff * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, nu_quadsum_pos_diff
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#XSec
ax2.step(bins, nu_xsec_neg_diff * -1
         , color = col_dict['Peach'] 
        )

ax2.step(bins, nu_xsec_pos_diff
         , color = col_dict['Peach'] 
        )
#-----------------------------------------------------------------#
#Flux
ax2.step(bins, nu_flx_neg_diff * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, nu_flx_pos_diff
         , color = col_dict['PastelGreen']
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, nu_stat_neg_diff * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, nu_stat_pos_diff
         , color = col_dict['Lavender']
        )

#-----------------------------------------------------------------#
#G4
ax2.step(bins, nu_g4_neg_diff * -1
         , color = col_dict['SkyBlue']
        )

ax2.step(bins, nu_g4_pos_diff
         , color = col_dict['SkyBlue']
        )
#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-2, 2)
ax2.axhline(y=1)
ax2.axhline(y=-1)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_combined_standard_error.png"), dpi=200)
    
plt.show()

<h3>Scale To POT ONLY</h3>

In [ ]:
scale_factor_nu = df_nu['scale_pot'].unique()[0]
print(scale_factor_nu)

In [ ]:
hnl_quadsum_neg_diff[1:] * hnl_dict['hnl_cv'] * scale_factor

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize = (6, 6), sharex = True)

xmin, xmax, xnbin = xmin, xmax, xnbin
xlimmin, xlimmax = xmin, xmax

#-----------------------------------------------------------------#
#central value
ax1.step(bins, nu_dict['nu_cv_plot'] * scale_factor_nu
         , color = col_dict['Flamingo'], label = r" $\nu$ Central Value")
#=========================================================================#
#QuadSum
ax1.errorbar(bins_mid, nu_dict['nu_cv'] * scale_factor_nu
            , [nu_quadsum_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
               , nu_quadsum_pos_diff[1:]  * scale_factor_nu * nu_dict['nu_cv']]
            , ls='none'
            , color = col_dict['DeepViolet']
            , capsize=8
            , label = "Quadrature Sum"
           )

#=========================================================================#
#STATS
ax1.errorbar(bins_mid, nu_dict['nu_cv']  * scale_factor_nu
            , [nu_stat_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
               ,nu_stat_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']]
            , ls='none'
            , color = col_dict['Lavender']
            , capsize=8
            , label = "Statistics"
           )
#=========================================================================#
#XSEC
ax1.errorbar(bins_mid, nu_dict['nu_cv']  * scale_factor_nu
            , [nu_xsec_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
               ,nu_xsec_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']]
            , ls='none'
            , color = col_dict['Peach']
            , capsize=8
            , label = r" $\nu$ XSec"
           )

#=========================================================================#
#FLUX
ax1.errorbar(bins_mid, nu_dict['nu_cv']  * scale_factor_nu
            , [nu_flx_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
               ,nu_flx_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']]
            , ls='none'
            , color = col_dict['PastelGreen']
            , capsize=8
            , label = "Flux"
           )

#=========================================================================#
#G4
ax1.errorbar(bins_mid, nu_dict['nu_cv']  * scale_factor_nu
            , [nu_g4_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
               ,nu_g4_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']]
            , ls='none'
            , color = col_dict['SkyBlue']
            , capsize=8
            , label = "Geant4 \nRe-Interactions"
           )
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)
plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]',  "Slices (No Scaling)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(nu_ymin, nu_ymax * scale_factor_nu)
#=========================================================================#
#-----------------------------------------------------------------#
#QuadSum
ax2.step(bins, nu_quadsum_neg_diff * -1
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )

ax2.step(bins, nu_quadsum_pos_diff
         , color = col_dict['DeepViolet'] , linestyle = '--'
        )
#-----------------------------------------------------------------#
#STATS
ax2.step(bins, nu_stat_neg_diff * -1
         , color = col_dict['Lavender']
        )

ax2.step(bins, nu_stat_pos_diff
         , color = col_dict['Lavender']
        )

#-----------------------------------------------------------------#
#XSec
ax2.step(bins, nu_xsec_neg_diff * -1
         , color = col_dict['Peach'] 
        )

ax2.step(bins, nu_xsec_pos_diff
         , color = col_dict['Peach'] 
        )
#-----------------------------------------------------------------#
#Flux
ax2.step(bins, nu_flx_neg_diff * -1
         , color = col_dict['PastelGreen']
        )

ax2.step(bins, nu_flx_pos_diff
         , color = col_dict['PastelGreen']
        )

#-----------------------------------------------------------------#
#G4
ax2.step(bins, nu_g4_neg_diff * -1
         , color = col_dict['SkyBlue']
        )

ax2.step(bins, nu_g4_pos_diff
         , color = col_dict['SkyBlue']
        )
#-----------------------------------------------------------------#

#ax2.legend(loc = 'upper right',fontsize = 14)
plot_tick(ax2, 16)
plot_title(ax2,"", 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]', "Fractional Error", 16)


ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-2, 2)
ax2.axhline(y=1)
ax2.axhline(y=-1)
#=========================================================================#
fig.tight_layout()

if ifSave:
    plt.savefig(savePath+str("nu_beam_bucket_combined_standard_error_scalePOT.png"), dpi=200)
    
plt.show()

In [ ]:
nu_dict['nu_cv_scale'] = nu_dict['nu_cv'] * scale_factor_nu

#quadSum
nu_dict['neg_quadSum_err_scale'] = nu_quadsum_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_quadSum_err_scale'] = nu_quadsum_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

#stat
nu_dict['neg_stat_err_scale'] = nu_stat_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_stat_err_scale'] = nu_stat_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

#stat
nu_dict['neg_stat_err_scale'] = nu_stat_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_stat_err_scale'] = nu_stat_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

#flux
nu_dict['neg_flx_err_scale'] = nu_flx_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_flx_err_scale'] = nu_flx_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

#g4
nu_dict['neg_g4_err_scale'] = nu_g4_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_g4_err_scale'] = nu_g4_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

#xsec
nu_dict['neg_xsec_err_scale'] = nu_xsec_neg_diff[1:] * scale_factor_nu * nu_dict['nu_cv']
nu_dict['pos_xsec_err_scale'] = nu_xsec_pos_diff[1:] * scale_factor_nu * nu_dict['nu_cv']

<h1> Quadrature Sum 2 Directional Error Bars Into 1 </h1>

<h3>HNL</h3>

In [ ]:
hnl_dict

In [ ]:
hnl_negpos_stat_arr = (hnl_dict['neg_stat_err_scale'], hnl_dict['pos_stat_err_scale'])
hnl_negpos_stat_arr = np.array(hnl_negpos_stat_arr)

hnl_dict['negpos_stat_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=hnl_negpos_stat_arr)

#------------------------------------------------------------------#
hnl_negpos_flx_arr = (hnl_dict['neg_flx_err_scale'], hnl_dict['pos_flx_err_scale'])
hnl_negpos_flx_arr = np.array(hnl_negpos_flx_arr)

hnl_dict['negpos_flx_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=hnl_negpos_flx_arr)

#------------------------------------------------------------------#
hnl_negpos_quadsum_arr = (hnl_dict['neg_quadSum_err_scale'], hnl_dict['pos_quadSum_err_scale'])
hnl_negpos_quadsum_arr = np.array(hnl_negpos_quadsum_arr)

hnl_dict['negpos_quadSum_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=hnl_negpos_quadSum_arr)

#------------------------------------------------------------------#

print('\n Stat')
print(hnl_negpos_stat_arr)
print(hnl_dict['negpos_stat_err_scale'])

print('\n Flux')
print(hnl_negpos_flx_arr)
print(hnl_dict['negpos_flx_err_scale'])

print('\n QuadSum')
print(hnl_negpos_quadSum_arr)
print(hnl_dict['negpos_quadSum_err_scale'])

<h3>Neutrino</h3>

In [ ]:
nu_dict.keys()

In [ ]:
nu_negpos_stat_arr = [nu_dict['neg_stat_err_scale'], nu_dict['pos_stat_err_scale']]
nu_negpos_stat_arr = np.array(nu_negpos_stat_arr)

nu_dict['negpos_stat_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_negpos_stat_arr)

#------------------------------------------------------------------#
nu_negpos_flx_arr = [nu_dict['neg_flx_err_scale'], nu_dict['pos_flx_err_scale']]
nu_negpos_flx_arr = np.array(nu_negpos_flx_arr)

nu_dict['negpos_flx_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_negpos_flx_arr)

#------------------------------------------------------------------#
nu_negpos_xsec_arr = [nu_dict['neg_xsec_err_scale'], nu_dict['pos_xsec_err_scale']]
nu_negpos_xsec_arr = np.array(nu_negpos_xsec_arr)

nu_dict['negpos_xsec_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_negpos_xsec_arr)

#------------------------------------------------------------------#
nu_negpos_g4_arr = [nu_dict['neg_g4_err_scale'], nu_dict['pos_g4_err_scale']]
nu_negpos_g4_arr = np.array(nu_negpos_g4_arr)

nu_dict['negpos_g4_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_negpos_g4_arr)

#------------------------------------------------------------------#
nu_negpos_quadSum_arr = [nu_dict['neg_quadSum_err_scale'], nu_dict['pos_quadSum_err_scale']]
nu_negpos_quadSum_arr = np.array(nu_negpos_quadSum_arr)

nu_dict['negpos_quadSum_err_scale'] = np.apply_along_axis(quadratureSumArray, axis=0, arr=nu_negpos_quadSum_arr)

#------------------------------------------------------------------#

print('\n Stat')
print(nu_negpos_stat_arr)
print(nu_dict['negpos_stat_err_scale'])

print('\n Flux')
print(nu_negpos_flx_arr)
print(nu_dict['negpos_flx_err_scale'])

print('\n XSec')
print(nu_negpos_xsec_arr)
print(nu_dict['negpos_xsec_err_scale'])

print('\n QuadSum')
print(nu_negpos_quadSum_arr)
print(nu_dict['negpos_quadSum_err_scale'])

<h1>Save Array</h1>

In [ ]:
hnl_dict.keys()

In [ ]:
nu_dict.keys()

In [ ]:
hnl_dict_save = {}
hnl_dict_save = {
    m :{ 
        'U': fitU,
        'bins': hnl_dict['bins'].tolist(),
        'sig': hnl_dict['hnl_cv_scale'].tolist(),
    
        'sig_stat': hnl_dict['pos_stat_err_scale'].tolist(),
    
        'sig_flx_lo': hnl_dict['hnl_cv_scale'] - hnl_dict['neg_flx_err_scale'].tolist(),
        'sig_flx_hi': hnl_dict['hnl_cv_scale'] + hnl_dict['pos_flx_err_scale'].tolist(),
        'sig_flx_quadSum': hnl_dict['negpos_flx_err_scale'].tolist(),

        'sig_quadSum_lo': (hnl_dict['hnl_cv_scale'] - hnl_dict['neg_quadSum_err_scale']).tolist(),
        'sig_quadSum_hi': (hnl_dict['hnl_cv_scale'] + hnl_dict['pos_quadSum_err_scale']).tolist(),
        'sig_quadSum': hnl_dict['negpos_quadSum_err_scale'].tolist(),
    
        'bkg': nu_dict['nu_cv_scale'].tolist(),
        
        'bkg_stat': nu_dict['pos_stat_err_scale'].tolist(),
        
        'bkg_flx_lo': nu_dict['nu_cv_scale'] - nu_dict['neg_flx_err_scale'].tolist(),
        'bkg_flx_hi': nu_dict['nu_cv_scale'] + nu_dict['pos_flx_err_scale'].tolist(),
        'bkg_flx_quadSum': nu_dict['negpos_flx_err_scale'].tolist(),
           
        'bkg_xsec_lo': nu_dict['nu_cv_scale'] - nu_dict['neg_xsec_err_scale'].tolist(),
        'bkg_xsec_hi': nu_dict['nu_cv_scale'] + nu_dict['pos_xsec_err_scale'].tolist(),
        'bkg_xsec_quadSum': nu_dict['negpos_xsec_err_scale'].tolist(),
        
        'bkg_g4_lo': nu_dict['nu_cv_scale'] - nu_dict['neg_g4_err_scale'].tolist(),
        'bkg_g4_hi': nu_dict['nu_cv_scale'] + nu_dict['pos_g4_err_scale'].tolist(),
        'bkg_g4_quadSum': nu_dict['negpos_g4_err_scale'].tolist(),
        
        'bkg_quadSum_lo': (nu_dict['nu_cv_scale'] - nu_dict['neg_quadSum_err_scale']).tolist(),
        'bkg_quadSum_hi': (nu_dict['nu_cv_scale'] + nu_dict['pos_quadSum_err_scale']).tolist(),
        'bkg_quadSum': nu_dict['negpos_quadSum_err_scale'].tolist()
        }
    
}

In [ ]:
for k, v in hnl_dict_save[m].items():
    print(k ,v)
    print('\n')

In [ ]:
np.save("../pkl_files/m"+str(m)+"_v3.npy", hnl_dict_save) 

###### 